# Recency, Frequency, Monetary Value analysis

**Datacamp** : https://app.datacamp.com/learn/courses/customer-segmentation-in-python

**RFM segmentation**

To do this, we are going to calculate three customer behavior metrics 
- Recency - which measures how recent was each customer's last purchase, 
- Frequency - which measures how many purchases the customer has done in the last 12 months, 
- MonetaryValue - measures how much has the customer spent in the last 12 months. 

We will use these values to assign customers to RFM segments.

**Grouping RFM values**

Next step is to group them into some sort of categorization such as high, medium and low. There are multiple ways to do that. 
- We can break customers into groups of equal size based on percentile values of each metric 
- We can assign either high or low value to each metric based on a 80/20% Pareto split 
- Or we can use existing knowledge from previous business insights about certain threshold values for each metric 


# Imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt

# RFM Segmentation

## Calculate percentile

In [24]:
# Create a simple DF
data = pd.DataFrame({'CustomerID': pd.Series(range(0,8)),
                     'Spend': [137, 335, 172, 355, 303, 233, 244, 229]
    
})
data

,CustomerID,Spend
0,0,137
1,1,335
2,2,172
3,3,355
4,4,303
5,5,233
6,6,244
7,7,229


In [25]:
# Calculate percentile
spend_quartiles = pd.qcut(x=data['Spend'], q=4, labels=range(1,5))
data['Spend_Quartiles'] = spend_quartiles
data.sort_values('Spend')

,CustomerID,Spend,Spend_Quartiles
0,0,137,1
2,2,172,1
7,7,229,2
5,5,233,2
6,6,244,3
4,4,303,3
1,1,335,4
3,3,355,4


## Assigning labels

In [26]:
# Create a simple DF
data = pd.DataFrame({'CustomerID': pd.Series(range(0,8)),
                     'Recency_Days': [37, 235, 396, 72, 255, 393, 203, 133]
    
})
data

,CustomerID,Recency_Days
0,0,37
1,1,235
2,2,396
3,3,72
4,4,255
5,5,393
6,6,203
7,7,133


In this case, the more recent the customer, the better !

In [27]:
# Create nb labels
r_labels = list(range(4,0,-1))
r_labels

[4, 3, 2, 1]

In [28]:
# Divide into groups based on quartiles
recency_quartiles = pd.qcut(x=data['Recency_Days'], q=4, labels=r_labels)

# Create new column
data['Recency_quartiles'] = recency_quartiles
data.sort_values('Recency_Days')

,CustomerID,Recency_Days,Recency_quartiles
0,0,37,4
3,3,72,4
7,7,133,3
6,6,203,3
1,1,235,2
4,4,255,2
5,5,393,1
2,2,396,1


The quartile labels are reversed, since the most recent customer are more valuable

## Custom labels

In [29]:
# Create string labels
r_labels = ['Active', 'Lapsed', 'Inactive', 'Churned']

# Divide into groups based on quartiles
recency_quartiles = pd.qcut(x=data['Recency_Days'], q=4, labels=r_labels)

# Create new column
data['Recency_Quartile'] = recency_quartiles

# Sort values
data.sort_values('Recency_Days')

,CustomerID,Recency_Days,Recency_quartiles,Recency_Quartile
0,0,37,4,Active
3,3,72,4,Active
7,7,133,3,Lapsed
6,6,203,3,Lapsed
1,1,235,2,Inactive
4,4,255,2,Inactive
5,5,393,1,Churned
2,2,396,1,Churned


# Calculating RFM metrics

In [30]:
# Load data
online_df = pd.read_csv('data/online.csv')
online_df.head()

,Unnamed: 0,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,416792,572558,22745,POPPY'S PLAYHOUSE BEDROOM,6,2011-10-25 08:26:00,2.10,14286,United Kingdom
1,482904,577485,23196,VINTAGE LEAF MAGNETIC NOTEPAD,1,2011-11-20 11:56:00,1.45,16360,United Kingdom
2,263743,560034,23299,FOOD COVER WITH BEADS SET 2,6,2011-07-14 13:35:00,3.75,13933,United Kingdom
3,495549,578307,72349B,SET/6 PURPLE BUTTERFLY T-LIGHTS,1,2011-11-23 15:53:00,2.10,17290,United Kingdom
4,204384,554656,21756,BATH BUILDING BLOCK WORD,3,2011-05-25 13:36:00,5.95,17663,United Kingdom


In [31]:
# Create TotalSum = Quantity * UnitPrice
online_df['TotalSum'] = online_df['Quantity'] * online_df['UnitPrice']
online_df = online_df.set_index('Unnamed: 0')
online_df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalSum
Unnamed: 0,,,,,,,,,
416792,572558,22745,POPPY'S PLAYHOUSE BEDROOM,6,2011-10-25 08:26:00,2.10,14286,United Kingdom,12.60
482904,577485,23196,VINTAGE LEAF MAGNETIC NOTEPAD,1,2011-11-20 11:56:00,1.45,16360,United Kingdom,1.45
263743,560034,23299,FOOD COVER WITH BEADS SET 2,6,2011-07-14 13:35:00,3.75,13933,United Kingdom,22.50
495549,578307,72349B,SET/6 PURPLE BUTTERFLY T-LIGHTS,1,2011-11-23 15:53:00,2.10,17290,United Kingdom,2.10
204384,554656,21756,BATH BUILDING BLOCK WORD,3,2011-05-25 13:36:00,5.95,17663,United Kingdom,17.85


In [32]:
online_df['InvoiceDate'] = pd.to_datetime(online_df['InvoiceDate'])
online_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 70864 entries, 416792 to 312243
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   InvoiceNo    70864 non-null  int64         
 1   StockCode    70864 non-null  object        
 2   Description  70864 non-null  object        
 3   Quantity     70864 non-null  int64         
 4   InvoiceDate  70864 non-null  datetime64[ns]
 5   UnitPrice    70864 non-null  float64       
 6   CustomerID   70864 non-null  int64         
 7   Country      70864 non-null  object        
 8   TotalSum     70864 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(3), object(3)
memory usage: 5.4+ MB


## Data preparation

In [33]:
# Which period
'Min: {}; Max: {}'.format(min(online_df.InvoiceDate),
                       max(online_df.InvoiceDate))

'Min: 2010-12-01 08:26:00; Max: 2011-12-09 12:49:00'

In [34]:
# Create a hypothetical snapshot_day data as if we're doing analysis recently
snapshot_date = max(online_df.InvoiceDate) + dt.timedelta(days=1)
snapshot_date

Timestamp('2011-12-10 12:49:00')

## Calculate RFM metrics

In [35]:
online_df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalSum
Unnamed: 0,,,,,,,,,
416792,572558,22745,POPPY'S PLAYHOUSE BEDROOM,6,2011-10-25 08:26:00,2.10,14286,United Kingdom,12.60
482904,577485,23196,VINTAGE LEAF MAGNETIC NOTEPAD,1,2011-11-20 11:56:00,1.45,16360,United Kingdom,1.45
263743,560034,23299,FOOD COVER WITH BEADS SET 2,6,2011-07-14 13:35:00,3.75,13933,United Kingdom,22.50
495549,578307,72349B,SET/6 PURPLE BUTTERFLY T-LIGHTS,1,2011-11-23 15:53:00,2.10,17290,United Kingdom,2.10
204384,554656,21756,BATH BUILDING BLOCK WORD,3,2011-05-25 13:36:00,5.95,17663,United Kingdom,17.85


- Recency - which measures how recent was each customer's last purchase,
- Frequency - which measures how many purchases the customer has done in the last 12 months,
- MonetaryValue - measures how much has the customer spent in the last 12 months.

In [36]:
# Aggregate data on a customer level
datamart = online_df.groupby(['CustomerID']).agg({
    'InvoiceDate': lambda x: (snapshot_date - x.max()).days,
    'InvoiceNo': 'count',
    'TotalSum': 'sum'
})

In [37]:
datamart.head()

,InvoiceDate,InvoiceNo,TotalSum
CustomerID,,,
12747,2,27,992.82
12748,1,967,7522.06
12749,4,37,813.45
12820,3,17,268.02
12822,71,9,146.15


In [38]:
# Rename columns
datamart.rename(columns={
    'InvoiceDate': 'Recency',
    'InvoiceNo': 'Frequency',
    'TotalSum': 'MonetaryValue'}, inplace=True)

In [39]:
datamart.head()

,Recency,Frequency,MonetaryValue
CustomerID,,,
12747,2,27,992.82
12748,1,967,7522.06
12749,4,37,813.45
12820,3,17,268.02
12822,71,9,146.15


In [40]:
# What average value ?
datamart.describe()

,Recency,Frequency,MonetaryValue
count,3701.000000,3701.000000,3701.000000
mean,94.298568,19.147257,380.535069
std,99.974969,44.831615,1474.285937
min,1.000000,1.000000,0.650000
25%,19.000000,4.000000,59.400000
50%,52.000000,9.000000,140.300000
75%,151.000000,21.000000,337.800000
max,374.000000,1532.000000,59596.070000


# Building RFM segments

**Recency**

Since the recency value measures the days since last transaction, we will rate customers who have been active more recently better than the less recent customers => lower is better

In [41]:
# Recency quartile
r_labels = range(4,0,-1)
r_quartiles = pd.qcut(x=datamart['Recency'], q=4, labels=r_labels)
datamart = datamart.assign(R = r_quartiles.values)
datamart.head()

,Recency,Frequency,MonetaryValue,R
CustomerID,,,,
12747,2,27,992.82,4
12748,1,967,7522.06,4
12749,4,37,813.45,4
12820,3,17,268.02,4
12822,71,9,146.15,2


**Frequency & Monetary**

The first difference is that the labels have a different order than recency, because frequency and monetary values are considered better when they are higher: we want customers to spend more and visit more often. Hence, we assign higher labels to higher values.

In [44]:
# Frequency and monetary quartiles
f_labels = range(1,5)
m_labels = range(1,5)

f_quartiles = pd.qcut(x=datamart['Frequency'], q=4, labels=f_labels)
m_quartiles = pd.qcut(x=datamart['MonetaryValue'], q=4, labels=m_labels)

datamart = datamart.assign(F = f_quartiles.values)
datamart = datamart.assign(M = m_quartiles.values)

In [46]:
datamart.head()

,Recency,Frequency,MonetaryValue,R,F,M
CustomerID,,,,,,
12747,2,27,992.82,4,4,4
12748,1,967,7522.06,4,4,4
12749,4,37,813.45,4,4,4
12820,3,17,268.02,4,3,3
12822,71,9,146.15,2,2,3


**Build RFM segment and RFM score**

Now the final step is to create the RFM Segment which is just a concatenated string of RFM values, and the RFM Score which is the sum of RFM values.

In [53]:
# Concatenate RFM quartile values : 
def join_rfm(x):
    return str(x['R']) + str(x['F']) + str(x['M'])

datamart['RFM_segment'] = datamart.apply(join_rfm, axis=1)

In [54]:
datamart['RFM_score'] = datamart[['R', 'F', 'M']].sum(axis=1)

In [55]:
datamart

,Recency,Frequency,MonetaryValue,R,F,M,RFM_segment,RFM_score
CustomerID,,,,,,,,
12747,2,27,992.82,4,4,4,444,12
12748,1,967,7522.06,4,4,4,444,12
12749,4,37,813.45,4,4,4,444,12
12820,3,17,268.02,4,3,3,433,10
12822,71,9,146.15,2,2,3,223,7
...,...,...,...,...,...,...,...,...
18280,278,2,38.70,1,1,1,111,3
18281,181,2,31.80,1,1,1,111,3
18282,8,2,30.70,4,1,1,411,6


This is the result we get - customers get 2 different variables that we created - an RFM segment based from three different RFM values, and the RFM score that sums up the RFM values and indicates a relative customer value.

# Analyzing RFM table

In [61]:
# Largest RFM segment (best practice to investigate size of the segment)
datamart.groupby('RFM_segment').size().sort_values(ascending=False)[:10]

RFM_segment
444    382
111    346
211    168
344    162
233    132
222    130
311    120
433    119
333    118
122    116
dtype: int64

In [63]:
# Filtering on RFM segments
datamart[datamart['RFM_segment']=='111'][:5]

,Recency,Frequency,MonetaryValue,R,F,M,RFM_segment,RFM_score
CustomerID,,,,,,,,
12837,173,2,10.55,1,1,1,111,3
12852,295,2,32.55,1,1,1,111,3
12902,265,4,42.03,1,1,1,111,3
12922,161,4,57.24,1,1,1,111,3
12929,312,3,42.90,1,1,1,111,3


In [65]:
# Summary metrics per RFM score
datamart.groupby('RFM_score').agg({
    'Recency': 'mean',
    'Frequency': 'mean',
    'MonetaryValue': ['mean', 'count']
}).round(1)

Recency Frequency MonetaryValue      
             mean      mean          mean count
RFM_score                                      
3           256.7       2.0          28.5   346
4           175.7       3.2          48.0   348
5           145.3       4.3          79.6   406
6           105.4       6.4         149.4   433
7            82.8       8.9         163.1   384
8            63.4      12.8         198.6   382
9            48.4      16.8         330.0   347
10           32.9      24.5         438.8   364
11           21.6      38.8         709.4   309
12            7.6      77.3        1709.0   382

The sizes are fairly similar across the RFM Score groups, and each of the **RFM values are better with the higher RFM Score segment**

In [66]:
# Grouping into named segments
def segment_me(df):
    if df['RFM_score'] >= 9:
        return 'Gold'
    elif (df['RFM_score'] >= 5) and (df['RFM_score'] < 9):
        return 'Silver'
    else:
        return 'Bronze'

In [67]:
datamart['GeneralSegment'] = datamart.apply(segment_me, axis=1)
datamart.head()

,Recency,Frequency,MonetaryValue,R,F,M,RFM_segment,RFM_score,GeneralSegment
CustomerID,,,,,,,,,
12747,2,27,992.82,4,4,4,444,12,Gold
12748,1,967,7522.06,4,4,4,444,12,Gold
12749,4,37,813.45,4,4,4,444,12,Gold
12820,3,17,268.02,4,3,3,433,10,Gold
12822,71,9,146.15,2,2,3,223,7,Silver


In [70]:
datamart.groupby('GeneralSegment').agg({
    'Recency': 'mean',
    'Frequency': 'mean',
    'MonetaryValue': ['mean', 'count']
}).round(1)

Recency Frequency MonetaryValue      
                  mean      mean          mean count
GeneralSegment                                      
Bronze           216.1       2.6          38.3   694
Gold              27.4      40.1         817.6  1402
Silver           100.1       8.0         146.7  1605

From the first look our segmentation does make sense. In reality it can take multiple takes of trial and error to find the right cut-offs.

In [74]:
# What is the average MonetaryValue for the segment with RFM_Score of 9 (nine)?
datamart[datamart['RFM_score']==9]['MonetaryValue'].mean()

330.0059106628242

In [75]:
datamart

,Recency,Frequency,MonetaryValue,R,F,M,RFM_segment,RFM_score,GeneralSegment
CustomerID,,,,,,,,,
12747,2,27,992.82,4,4,4,444,12,Gold
12748,1,967,7522.06,4,4,4,444,12,Gold
12749,4,37,813.45,4,4,4,444,12,Gold
12820,3,17,268.02,4,3,3,433,10,Gold
12822,71,9,146.15,2,2,3,223,7,Silver
...,...,...,...,...,...,...,...,...,...
18280,278,2,38.70,1,1,1,111,3,Bronze
18281,181,2,31.80,1,1,1,111,3,Bronze
18282,8,2,30.70,4,1,1,411,6,Silver


In [76]:
# Export datamart in a new csv
datamart.to_csv('data/datamart_RFM.csv', index=False)